In [1]:
import kagglehub
import pandas as pd
import os

# Download latest version
path = kagglehub.dataset_download("rasho330/job-application-email-anonymized-and-feature-rich")

print("Path to dataset files:", path)

# Get the CSV file path from the downloaded dataset directory
file_path = os.path.join(path, os.listdir(path)[0])
df = pd.read_csv(file_path)


/Users/cm/Downloads/email_ingestion/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/cm/.cache/kagglehub/datasets/rasho330/job-application-email-anonymized-and-feature-rich/versions/2


In [12]:
Label_1 = df['email_body']
Label_1 = Label_1.dropna().unique()
Label_1.shape

(464,)

In [3]:
path = kagglehub.dataset_download("marcelwiechmann/enron-spam-data")
print("Path to dataset files:", path)

file_path = os.path.join(path, os.listdir(path)[0])
df_2 = pd.read_csv(file_path)

Path to dataset files: /Users/cm/.cache/kagglehub/datasets/marcelwiechmann/enron-spam-data/versions/3


In [14]:
# Filter out null messages
df_2_clean = df_2[df_2['Message'].notna()]

# Separate spam and ham
spam_messages = df_2_clean[df_2_clean['Spam/Ham'] == 'spam'].sample(n=464, random_state=42)
ham_messages = df_2_clean[df_2_clean['Spam/Ham'] == 'ham'].sample(n=464, random_state=42)

# Combine them
Label_0 = pd.concat([spam_messages['Message'], ham_messages['Message']]).reset_index(drop=True)
Label_0.shape


(928,)

In [18]:
print(Label_0.sample(n = 10))

744    i ' m not familiar with the individual assets ...
812    start date : 1 / 16 / 02 ; hourahead hour : 16...
198    fyi , kim .\n- - - - - original message - - - ...
782    please find below the weekly project status up...
346    start date : 2 / 6 / 02 ; hourahead hour : 24 ...
672    - - - - - original message - - - - -\nfrom : s...
748    sally ,\nattached is the detail of the analyst...
645    darren :\nena was invoiced by coral energy for...
697    - - - - - - - - - - - - - - - - - - - - - - fo...
382    start date : 2 / 6 / 02 ; hourahead hour : 24 ...
Name: Message, dtype: str


In [22]:
# Concatenate Label_0 and Label_1 into a single DataFrame with a label column
df_label_0 = pd.DataFrame({'text': Label_0, 'is_job_related': 0})
df_label_1 = pd.DataFrame({'text': Label_1, 'is_job_related': 1})

combined_df = pd.concat([df_label_0, df_label_1], ignore_index=True)
type(combined_df.sample(n=10))

pandas.DataFrame

In [25]:
sample_df = combined_df.sample(n=10, random_state=42).reset_index(drop=True)

for i, row in sample_df.iterrows():
    print(f"\n===== #{i} | is_job_related={row['is_job_related']} =====")
    print(row["text"])



===== #0 | is_job_related=1 =====
Hi Michael Gary Scott,
 
Thank you for your interest in Flip! We wanted to let you know we received your application for Data Scientist, and we are delighted that you would consider joining our team.
 
Our team will review your application and will be in touch if your qualifications match our needs for the role. If you are not selected for this position, keep an eye on our jobs page as we're growing and adding openings.
 
Best,
The Flip Team

===== #1 | is_job_related=0 =====
louise ,
thanks so much for your speedy reply . i will pass your comments on to dave walker in media relations .
i included kevin because he is listed with you , greg and john on a " management " slide . if there are others who should be
included on greg ' s core management team for the announcement , please let me know who they are and their titles . please
also let me know if it is inappropriate to include kevin . if there is an org chart , that might be helpful , too . it woul

In [26]:
# Display basic dataset information
print("="*80)
print("DATASET OVERVIEW")
print("="*80)
print(f"\nDataset shape: {combined_df.shape}")
print(f"Total samples: {len(combined_df)}")
print(f"\nColumn names and types:")
print(combined_df.dtypes)

# Check for missing values
print("\n" + "="*80)
print("MISSING VALUES")
print("="*80)
print(combined_df.isnull().sum())
print(f"Missing values percentage:\n{(combined_df.isnull().sum() / len(combined_df) * 100).round(2)}")

# Class distribution
print("\n" + "="*80)
print("CLASS DISTRIBUTION")
print("="*80)
print(combined_df['is_job_related'].value_counts())
print(f"\nClass distribution (%):")
print((combined_df['is_job_related'].value_counts(normalize=True) * 100).round(2))

# Text statistics
print("\n" + "="*80)
print("TEXT STATISTICS")
print("="*80)
combined_df['text_length'] = combined_df['text'].astype(str).apply(len)
combined_df['word_count'] = combined_df['text'].astype(str).apply(lambda x: len(x.split()))

print(f"\nCharacter count statistics:")
print(combined_df['text_length'].describe())
print(f"\nWord count statistics:")
print(combined_df['word_count'].describe())

print(f"\nText length by class:")
print(combined_df.groupby('is_job_related')[['text_length', 'word_count']].describe().round(2))

# Sample inspection
print("\n" + "="*80)
print("SAMPLE INSPECTION")
print("="*80)
print(f"\nJob-related email sample (is_job_related=1):")
print(combined_df[combined_df['is_job_related'] == 1]['text'].iloc[0][:500])
print(f"\nNon job-related email sample (is_job_related=0):")
print(combined_df[combined_df['is_job_related'] == 0]['text'].iloc[0][:500])

DATASET OVERVIEW

Dataset shape: (1392, 2)
Total samples: 1392

Column names and types:
text                str
is_job_related    int64
dtype: object

MISSING VALUES
text              0
is_job_related    0
dtype: int64
Missing values percentage:
text              0.0
is_job_related    0.0
dtype: float64

CLASS DISTRIBUTION
is_job_related
0    928
1    464
Name: count, dtype: int64

Class distribution (%):
is_job_related
0    66.67
1    33.33
Name: proportion, dtype: float64

TEXT STATISTICS

Character count statistics:
count     1392.000000
mean      1296.738506
std       2518.032197
min         14.000000
25%        541.750000
50%        839.500000
75%       1180.000000
max      68196.000000
Name: text_length, dtype: float64

Word count statistics:
count     1392.000000
mean       270.890086
std        505.726906
min          3.000000
25%         94.000000
50%        151.000000
75%        325.000000
max      13428.000000
Name: word_count, dtype: float64

Text length by class:
         

In [27]:
def clip_email(text: str, head: int = 3000, tail: int = 1000) -> str:
    text = str(text).strip()
    if len(text) <= head + tail:
        return text
    return text[:head] + "\n\n[TRUNCATED]\n\n" + text[-tail:]

combined_df["text_clipped"] = combined_df["text"].apply(clip_email)

In [28]:
import re

def normalize_ws(s: str) -> str:
    s = str(s)
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

combined_df["text_clipped"] = combined_df["text_clipped"].apply(normalize_ws)


In [ ]:
import pandas as pd

df = combined_df.copy()

df["Text"] = df["text"].astype(str).str.strip()
df["Tags"] = df["is_job_related"].map({1: "YES_TRACK", 0: "NO_IGNORE"})

out = df[["Text", "Tags"]]
out.to_csv("labeling/train_dataverse.csv", index=False, encoding="utf-8")
print(out["Tags"].value_counts())


Tags
NO_IGNORE    928
YES_TRACK    464
Name: count, dtype: int64
